In [3]:
import pandas as pd

In [17]:
data = pd.read_csv('C:\\Users\\Predator\\Downloads\\hpt_small.csv')

In [18]:
data.head()

,age,wc,education,marital status,race,gender,hours per week,IncomeClass
0,38,Private,HS-grad,Divorced,White,Male,40,<=50K
1,28,Private,Bachelors,Married,Black,Female,40,<=50K
2,37,Private,Masters,Married,White,Female,40,<=50K
3,31,Private,Masters,Never-married,White,Female,50,>50K
4,42,Private,Bachelors,Married,White,Male,40,>50K


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             500 non-null    int64 
 1   wc              500 non-null    object
 2   education       500 non-null    object
 3   marital status  500 non-null    object
 4   race            500 non-null    object
 5   gender          500 non-null    object
 6   hours per week  500 non-null    int64 
 7   IncomeClass     500 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [20]:
data_prep = pd.get_dummies(data,drop_first = True)

In [21]:
data_prep.head()

,age,hours per week,wc_ Local-gov,wc_ Private,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital status_ Never-married,marital status_ Widowed,marital status_Married,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,gender_ Male,IncomeClass_ >50K
0,38,40,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False
1,28,40,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False
2,37,40,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False
3,31,50,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,True
4,42,40,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True


In [9]:
data_prep.describe()

,age,hours per week
count,500.000000,500.000000
mean,36.510000,40.664000
std,12.452793,10.485561
min,18.000000,6.000000
25%,27.000000,40.000000
50%,34.000000,40.000000
75%,44.000000,45.000000
max,90.000000,80.000000


In [10]:
from sklearn.svm import SVC

In [11]:
svc = SVC(random_state = 1234)

In [12]:
X = data_prep.iloc[:, :-1]
y = data_prep.iloc[:, -1]

In [16]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

## Grid Search

In [10]:
from sklearn.model_selection import GridSearchCV

In [ ]:
## Incredible amount of time...

svc_param = {'C':[0.01,0.1,0.5,1,2,5,10],
             'kernel':['rbf','linear','poly'],
             'gamma':[0.1,0.25,0.5,1,5]
            }

svc_grid = GridSearchCV(estimator = svc,
                       param_grid = svc_param,
                       scoring = 'accuracy',
                       cv = 10,
                       n_jobs = -1,
                       return_train_score = True)

svc_grid_fit = svc_grid.fit(X, y)

cv_results_svc = pd.DataFrame.from_dict(svc_grid_fit.cv_results_)

svc_top_rank = cv_results_svc[cv_results_svc['rank_test_score'] == 1]

## Random Search

In [23]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1234)

In [24]:
from sklearn.model_selection import RandomizedSearchCV

In [26]:
rfc_param = {'n_estimators':[5,10,15,20,30,40],
            'min_samples_split':[8,16,32,64],
            'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]}

rfc_rs = RandomizedSearchCV(estimator = rfc,
                           param_distributions = rfc_param,
                           scoring = 'accuracy',
                           cv = 10,
                           n_iter = 10,
                           return_train_score = True,
                           random_state = 1234)

rfc_rs_fit = rfc_rs.fit(X_train, y_train)

In [27]:
cv_results_rfc_rs = pd.DataFrame.from_dict(rfc_rs_fit.cv_results_)

rfc_rs_top_rank = cv_results_rfc_rs[cv_results_rfc_rs['rank_test_score'] == 1]

print('\n\n')
print('Random Forest Classifier')
print(rfc_rs_top_rank['mean_test_score'])
print(rfc_rs_top_rank['mean_train_score'])

# Print the best parameters of Randomized Search for Random Forest
print('\n The best Parameters are : ')
print(rfc_rs_fit.best_params_)




Random Forest Classifier
4    0.835
Name: mean_test_score, dtype: float64
4    0.888889
Name: mean_train_score, dtype: float64

 The best Parameters are : 
{'n_estimators': 10, 'min_samples_split': 16, 'min_samples_leaf': 1}


## Bayesian Optimisation(chetin)

In [29]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [30]:
opt = BayesSearchCV(
    SVC(),
    {
        'C': Real(0.01, 10, prior = 'log-uniform'),
        'kernel': Categorical(['linear','poly','rbf'])
    },
    n_iter = 100,
    random_state = 0
)

In [31]:
_ = opt.fit(X_train, y_train)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations